# LLM Showcase with Shakespeare in German

In [1]:
#for colab only
#for problems with package versions look in the requirements-colab-freeze.txt and requirements-pip-freeze.txt files
!pip install tiktoken
!git clone https://github.com/phonosync/demo_llm
!mv GenAI_LLM/* ./
import gdown
file_id = '19caXJPPRXEHm18Y5L5YI0erfCSaYbO4T'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'out-shakespeare-deutsch/ckpt_pretrained.pt'

gdown.download(url, output, quiet=False)

In [ ]:
import os
import tiktoken
from contextlib import nullcontext
import torch
from model import GPTConfig, GPT

# -----------------------------------------------------------------------------
out_dir = 'out-shakespeare-deutsch'
num_samples = 3 # number of samples to draw
max_new_tokens = 200 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
seed = 1337
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
device = 'cpu'
dtype = 'float16'
# -----------------------------------------------------------------------------

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext()

# model
ckpt_path = os.path.join(out_dir, 'ckpt_pretrained.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
gptconf = GPTConfig(**checkpoint['model_args'])
model = GPT(gptconf)
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)

model.eval()
model.to(device)

enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)

number of parameters: 29.94M


In [6]:
start = "Dieser Satz wird als Anfang genutzt " # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"

start_ids = encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            print('---------------')
            print(decode(y[0].tolist()))

---------------
Dieser Satz wird als Anfang genutzt erscheinen,
Und glaubt, daß der Tod ein Feind
Und seiner Ehre stets und das schlimmste
Kann ich doch und weiß, wie man ihn vergessen.

ROSSE
Die Hände, die Wohl verdient,
So waren sie, den Tag vor dem Tod,
Und widersprach das Tod des Anblick.
Die Sicherheit stolzer wahren, wird von ihm
Beim Himmel, in Vermätern, wo er sich
Den Vorsatz des Dunsinan dem unserm Grames Haupt,
Wo sein so in der Natur, mit ihm verschworen,
Wo er verhobt, daß er stirbt.

MACBETH
Gift er so,
---------------
Dieser Satz wird als Anfang genutzt erschlagen.

MALCOLM
Noch soll er nicht, wenn man ihn dran,
Denn unverzückt, als er die Sünden
Auf unsre Sache, daß unser Schicksal
Gewinn.--Haben auch den Staat!

MACBETH
Wäre darüber zu spüten.

LADY MACDUFF
Ich will dich nicht stehn,
Daß es erscheint; doch wie er kann,
Worin, und wie sie im Nacht verriet.

MACBETH
Es tut gut, wenn er ihm zu sagen.

MACBETH
Ich haß es tun, daß man ihm folgen.

MACBETH
Jetzt, wie er auf